In [ ]:
import matplotlib.pyplot as plt
import requests_cache
import moonbox

In [ ]:
year = 2025
session = requests_cache.CachedSession("cache")

In [ ]:
# get the calendar
calendar = moonbox.calendar(year=year, session=session)


# choose which dates to highlight with their month: first of the solar year, first
# of a lunar month, or first of a solar month
def show_month(x):
    return (
        (x["date"].month == 1 and x["date"].day == 1)
        or x["lunar_day"] == 0
        or x["date"].day == 1
    )


calendar = [x | {"show_month": show_month(x)} for x in calendar]

# but we don't want to show two months in a row, if they are physically adjacent
for i in range(1, len(calendar)):
    if (
        calendar[i]["show_month"]
        and calendar[i - 1]["show_month"]
        and calendar[i]["lunar_month"] == calendar[i - 1]["lunar_month"]
    ):
        calendar[i]["show_month"] = False

In [ ]:
plt.ioff()

# set up the calendar geometry
canvas_size = (28.0, 22.0)

calendar_width_prop = 0.95
calendar_middle = 0.7  # middle of calendar is this fraction down the canvas

spacer_x_prop = 0.3
spacer_y_prop = 1.6

date_font_size = 20
date_spacer = 0.1  # proportion of radius

title_y_prop = 0.05  # title is this fraction down the canvas
title_font_size = 60

signature_font_size = 30
signature_spacer_x_prop = 0.01
signature_spacer_y_prop = 0.01

font_family = "serif"

# data
n_months = calendar[-1]["lunar_month"] + 1
n_days = 30  # maximum length of a lunar month

# derived positions
calendar_width = canvas_size[0] * calendar_width_prop
moon_radius = calendar_width / (2 * n_days + spacer_x_prop * (n_days - 1))
spacer_x = spacer_x_prop * moon_radius
spacer_y = spacer_y_prop * moon_radius

calendar_height = n_months * 2 * moon_radius + (n_months - 1) * spacer_y

calendar_x = (canvas_size[0] - calendar_width) / 2
calendar_y = canvas_size[1] - (canvas_size[1] - calendar_height) * calendar_middle

fig = plt.figure(figsize=canvas_size)
ax = fig.add_axes([0, 0, 1, 1])
assert isinstance(ax, plt.Axes)  # to improve type hints
ax.set_xlim(0, canvas_size[0])
ax.set_ylim(0, canvas_size[1])

ax.set_facecolor("black")

for datum in calendar:
    if datum["show_month"]:
        date_label = datum["date"].strftime("%b %-d")
        date_fontweight = "bold"
    else:
        date_label = datum["date"].strftime("%-d")
        date_fontweight = "normal"

    # draw the moon
    f = datum["illumination"] / 100
    direction = datum["phase"].split()[0].lower()

    moon_x = (
        calendar_x + (2 * moon_radius + spacer_x) * datum["lunar_day"] + moon_radius
    )
    # note that y decreases as months increase
    moon_y = (
        calendar_y - (2 * moon_radius + spacer_y) * datum["lunar_month"] - moon_radius
    )

    moonbox.draw_moon(
        ax,
        x=moon_x,
        y=moon_y,
        radius=moon_radius,
        f=f,
        direction=direction,
        dark="0.2",
    )

    # date labels
    ax.text(
        x=moon_x,
        y=moon_y + (1.0 + date_spacer) * moon_radius,
        s=date_label,
        horizontalalignment="center",
        verticalalignment="bottom",
        color="white",
        fontsize=date_font_size,
        fontweight=date_fontweight,
        fontfamily=font_family,
    )

# title
ax.text(
    x=canvas_size[0] / 2,
    y=canvas_size[1] * (1.0 - title_y_prop),
    s=f"Lunar Months of {year}",
    verticalalignment="top",
    horizontalalignment="center",
    fontsize=title_font_size,
    fontweight="bold",
    fontfamily=font_family,
    color="white",
)

# signature
ax.text(
    x=canvas_size[0] * (1.0 - signature_spacer_x_prop),
    y=canvas_size[1] * signature_spacer_y_prop,
    s="Scott Olesen",
    verticalalignment="bottom",
    horizontalalignment="right",
    fontsize=signature_font_size,
    fontfamily=font_family,
    color="white",
)

# do not show the figure; rendering in VSCode is very slow
fig.savefig("lunar_calendar_quick.png", dpi=72)

In [ ]:
# production quality
fig.savefig("lunar_calendar.png", dpi=300)
fig.savefig("lunar_calendar.pdf")